# YouTube Video Summarizer using Groq

This notebook demonstrates how to create a YouTube video summarizer that:
1. Takes a YouTube URL as input
2. Extracts the video's transcription
3. Uses Groq to generate a concise summary

## Setup and Requirements
First, we'll install the required packages:

In [10]:
!pip install youtube-transcript-api groq python-dotenv -q

## Import Required Libraries

In [11]:
from youtube_transcript_api import YouTubeTranscriptApi
import groq
import os
from dotenv import load_dotenv
from urllib.parse import urlparse, parse_qs

# Load environment variables
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')  # Modify this to your API key

# Initialize Groq client
client = groq.Groq(
    api_key=groq_api_key  # Put your API key here
)

## Helper Functions

In [12]:
def extract_video_id(url):
    """Helper function to extract YouTube video ID from URL"""
    parsed_url = urlparse(url)
    if parsed_url.hostname == 'youtu.be':
        return parsed_url.path[1:]
    if parsed_url.hostname in ('www.youtube.com', 'youtube.com'):
        if parsed_url.path == '/watch':
            return parse_qs(parsed_url.query)['v'][0]
    return None

def get_transcript(video_id):
    """Helper function to get transcript for a YouTube video"""
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['en', 'es'])
        return ' '.join([t['text'] for t in transcript_list])
    except Exception as e:
        print(f"Error getting transcript: {e}")
        return None

def summarize_text(text, language="spanish"):
    """Summarize text using Groq"""
    prompt = f"""Please provide a comprehensive summary of the following video transcript in {language}. 
    Focus on the main points, key insights, and important conclusions:

    {text}

    Please structure the summary with:
    1. Main Topic/Theme
    2. Key Points
    3. Important Details
    4. Conclusions"""

    try:
        completion = client.chat.completions.create(
            model="llama-3.2-11b-vision-preview",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=2048
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error in summarization: {e}")
        return None

## Main Function to Summarize YouTube Video

In [23]:
def summarize_youtube_video(url, language="spanish"):
    """Main function to summarize a YouTube video"""
    # Extract video ID
    video_id = extract_video_id(url)
    if not video_id:
        return "Invalid YouTube URL"
    
    # Get transcript
    transcript = get_transcript(video_id)
    transcript = transcript[:6000*4]
    if not transcript:
        return "Could not retrieve transcript"
    
    # Generate summary
    summary = summarize_text(transcript, language)
    if not summary:
        return "Could not generate summary"
    
    return summary

## Example Usage

To use this summarizer, you'll need to:
1. Create a `.env` file in the same directory as this notebook
2. Add your Groq API key to the `.env` file: `GROQ_API_KEY=your_api_key_here`

Then you can use the summarizer as shown below:

In [24]:
# Example usage
youtube_url = "https://www.youtube.com/watch?v=r5mEZCiXYN4&pp=ygUMZXBtIGNvbG9tYmlh"
language = "spanish"
summary = summarize_youtube_video(youtube_url, language)
print(summary)

**1. Main Topic/Theme**

La emergencia invernal en Colombia, específicamente en los departamentos de Chocó, Guajira, Santander y Bogotá, ha causado daños significativos y afectado a miles de personas.

**2. Key Points**

* La emergencia invernal ha causado daños en varios departamentos de Colombia, incluyendo Chocó, Guajira, Santander y Bogotá.
* Miles de personas han sido afectadas, con algunas familias sin acceso a alimentos, agua potable y atención médica.
* La falta de infraestructura y servicios básicos en algunas regiones ha agravado la situación.
* La ayuda humanitaria ha sido lenta en llegar a algunas áreas, lo que ha generado preocupación y angustia entre los afectados.
* La solidaridad de la comunidad colombiana ha sido notable, con donaciones y ayudas llegando a los afectados.

**3. Important Details**

* En el departamento de Chocó, 188.000 personas han sido afectadas, con 40.000 familias damnificadas.
* En la alta Guajira, 192.000 personas han sido afectadas, con una sola 